# PyWry 2.0 Demonstration

This notebook demonstrates the capabilities of PyWry 2.0, showcasing:
- **Rendering Modes**:
  - **Notebook**:
      - Comparison of `anywidget` vs `InlineWidget` (IFrame) paths.
  - **Native Window**:
      - Open the same content in a native window (Must be on a desktop OS)
- **Abstracted Callback API** (Toolbars & Events)
- **Plotly** integration
- **AG Grid** integration
- **Theme Toggling** via custom buttons
- **Alert Events** Render customizable toast notifications 

## Prerequisites

You should have the `pywry` library installed, as well as (only required for this demo):
- AnyWidget
- Plotly
- Pandas

In [ ]:
import contextlib
from unittest.mock import patch
import pandas as pd
import plotly.express as px
import pywry.widget

# Use high-level functions
from pywry.inline import show_dataframe, show_plotly

# Check environment
print(f"Has AnyWidget: {pywry.widget.HAS_ANYWIDGET}")

## 1. Data Preparation

We'll use a standard dataset for both the Chart and the Grid.

In [ ]:
# Create sample data
df = px.data.gapminder().query("country=='Canada'")
columns = df.columns.tolist()
data = df.to_dict(orient="records")


def get_figure(theme="plotly_dark"):
    """Create the figure with a specific theme template."""
    return px.bar(df, x="year", y="pop", title="Canada Population", template=theme)

## 2. Shared Theme Logic

We define a callback handler that toggles the theme. This handler works for **all** rendering paths because PyWry abstracts the event mechanism.

The Logic:
1.  Receive event from 'Toggle Theme' button.
2.  Check current state (we store it on the widget object for convenience).
3.  Emit `pywry:update-theme` to the frontend.

In [ ]:
# Theme state tracking
theme_states = {
    "aw_plotly": "dark",
    "aw_grid": "dark",
    "iframe_plotly": "dark",
    "iframe_grid": "dark",
}


# Define toolbar configurations
def make_toolbar(position: str = "top"):
    """Create a toolbar with a toggle theme button."""
    return {
        "position": position,
        "items": [
            {"type": "button", "label": "Toggle Theme", "event": "custom:toggle"}
        ],
    }


# --- Handlers ---
# Note: These handlers rely on the widget instances being defined in the global scope
# (aw_plotly, aw_grid, etc.) at the time the callback is executed.
# Event names must match: "custom:toggle"


def toggle_theme_plotly(data, event_type, label):
    """Callback for AnyWidget Plotly."""
    theme_states["aw_plotly"] = (
        "light" if theme_states["aw_plotly"] == "dark" else "dark"
    )
    new_template = (
        "plotly_white" if theme_states["aw_plotly"] == "light" else "plotly_dark"
    )
    aw_plotly.emit("pywry:update-theme", {"theme": new_template})


def toggle_theme_grid(data, event_type, label):
    """Callback for AnyWidget Grid."""
    theme_states["aw_grid"] = "light" if theme_states["aw_grid"] == "dark" else "dark"
    ag_theme = (
        "ag-theme-alpine"
        if theme_states["aw_grid"] == "light"
        else "ag-theme-alpine-dark"
    )
    aw_grid.emit("pywry:update-theme", {"theme": ag_theme})


def toggle_theme_iframe_plotly(data, event_type, label):
    """Callback for IFrame Plotly."""
    theme_states["iframe_plotly"] = (
        "light" if theme_states["iframe_plotly"] == "dark" else "dark"
    )
    new_template = (
        "plotly_white" if theme_states["iframe_plotly"] == "light" else "plotly_dark"
    )
    iframe_plotly.emit("pywry:update-theme", {"theme": new_template})


def toggle_theme_iframe_grid(data, event_type, label):
    """Callback for IFrame Grid."""
    theme_states["iframe_grid"] = (
        "light" if theme_states["iframe_grid"] == "dark" else "dark"
    )
    ag_theme = (
        "ag-theme-alpine"
        if theme_states["iframe_grid"] == "light"
        else "ag-theme-alpine-dark"
    )

    iframe_grid.emit("pywry:update-theme", {"theme": ag_theme})

## 3. AnyWidget Architecture (Modern)

This uses the modern `anywidget` library (if installed). It provides tighter integration with Jupyter/VS Code, bidirectional communication without a local server port, and better performance.

**Note:** `show_plotly` and `show_dataframe` default to this mode if available.

In [ ]:
print("--- Plotly via AnyWidget (Default) ---")

aw_plotly = show_plotly(
    figure=get_figure("plotly_dark"),
    title="AnyWidget Plotly",
    toolbars=[make_toolbar("top")],
    callbacks={"custom:toggle": toggle_theme_plotly},  # Must use namespace:event format
)

In [ ]:
print("--- AG Grid via AnyWidget (Default) ---")

# Register callback directly
aw_grid = show_dataframe(
    df=data,
    title="AnyWidget Grid",
    toolbars=[make_toolbar("bottom")],
    callbacks={"custom:toggle": toggle_theme_grid},  # Must use namespace:event format
)

## 4. IFrame Architecture (Notebook Fallback or Standalone)

This uses a local FastAPI server running in a background thread and serves content via an `<iframe>`. This is used when `anywidget` is not available or for standalone usage.

We use a helper context manager to temporarily disable `HAS_ANYWIDGET` to simulate this environment.

In [ ]:
@contextlib.contextmanager
def force_iframe_mode():
    """Temporarily mock pywry.widget.HAS_ANYWIDGET to False."""
    with patch("pywry.widget.HAS_ANYWIDGET", False):
        yield


print("--- Plotly via IFrame (FastAPI) ---")

with force_iframe_mode():
    iframe_plotly = show_plotly(
        figure=get_figure("plotly_dark"),
        title="IFrame Plotly",
        toolbars=[make_toolbar("top")],
        callbacks={
            "custom:toggle": toggle_theme_iframe_plotly
        },  # Must use namespace:event format
    )

In [ ]:
print("--- AG Grid via IFrame (FastAPI) ---")

with force_iframe_mode():
    iframe_grid = show_dataframe(
        df=data,
        title="IFrame Grid",
        toolbars=[make_toolbar("bottom")],
        callbacks={
            "custom:toggle": toggle_theme_iframe_grid
        },  # Must use namespace:event format
    )

## 5. Native Window Architecture

This uses the native PyWry window (Tauri/WebView2) for standalone desktop rendering. The content opens in a separate native window outside of the notebook.

**Note:** Native windows require running outside of an IFrame context and use the `PyWry` class directly.

In [ ]:
from pywry import PyWry, WindowMode, ThemeMode
from pywry import runtime  # For emitting events to native windows


# Context manager to force native window mode (bypass notebook detection)
@contextlib.contextmanager
def force_native_window_mode():
    """Temporarily mock should_use_inline_rendering to return False."""
    with patch("pywry.app.should_use_inline_rendering", return_value=False):
        yield


# Theme state for native windows
native_theme_states = {
    "native_plotly": "dark",
    "native_grid": "dark",
}


# Native window toolbar - using same format as notebook but with namespace:event format
def make_native_toolbar(position: str = "top"):
    """Create a toolbar for native windows."""
    return {
        "position": position,
        "items": [
            {"type": "button", "label": "Toggle Theme", "event": "custom:toggle"}
        ],
    }


# Create PyWry app in NEW_WINDOW mode (each show creates a new window)
app = PyWry(
    mode=WindowMode.NEW_WINDOW,
    theme=ThemeMode.DARK,
    title="PyWry Native Demo",
)

print(f"PyWry app initialized: mode={app._mode_enum.value}, theme={app.theme.value}")

In [ ]:
print("--- Plotly via Native Window ---")


# Define toggle callback for native Plotly
def toggle_theme_native_plotly(data, event_type, label):
    """Callback for Native Window Plotly."""
    native_theme_states["native_plotly"] = (
        "light" if native_theme_states["native_plotly"] == "dark" else "dark"
    )
    new_template = (
        "plotly_white"
        if native_theme_states["native_plotly"] == "light"
        else "plotly_dark"
    )

    # For native windows, use runtime.emit_event() instead of widget.emit()
    app.emit("pywry:update-theme", {"theme": new_template}, str(native_plotly_label))


# Show Plotly in native window (bypass notebook detection)
with force_native_window_mode():
    native_plotly_label = app.show_plotly(
        figure=get_figure("plotly_dark"),
        title="Native Window Plotly",
        toolbars=[make_native_toolbar("top")],
        callbacks={"custom:toggle": toggle_theme_native_plotly},
    )

print(f"Native Plotly window created: {native_plotly_label}")

In [ ]:
print("--- AG Grid via Native Window ---")


# Define toggle callback for native Grid
def toggle_theme_native_grid(data, event_type, label):
    """Callback for Native Window Grid."""
    native_theme_states["native_grid"] = (
        "light" if native_theme_states["native_grid"] == "dark" else "dark"
    )
    ag_theme = (
        "ag-theme-alpine"
        if native_theme_states["native_grid"] == "light"
        else "ag-theme-alpine-dark"
    )
    # For native windows, use runtime.emit_event() instead of widget.emit()
    app.emit(
        "pywry:update-theme",
        {"theme": ag_theme},
        str(native_grid_label),
    )


# Show AG Grid in native window (bypass notebook detection)
with force_native_window_mode():
    native_grid_label = app.show_dataframe(
        data=data,
        title="Native Window Grid",
        toolbars=[make_native_toolbar("bottom")],
        callbacks={"custom:toggle": toggle_theme_native_grid},
    )

print(f"Native Grid window created: {native_grid_label}")

## 6. Toast Notifications (pywry:alert)

PyWry provides a unified toast notification system that works consistently across all rendering paths. The alert system supports 5 types:

| Type | Icon | Behavior |
|------|------|----------|
| `info` | ℹ️ | Auto-dismiss after 5s |
| `success` | ✅ | Auto-dismiss after 3s |
| `warning` | ⚠️ | Persists until clicked |
| `error` | ⛔ | Persists until clicked |
| `confirm` | ❓ | Shows Cancel/Confirm buttons |

**Keyboard shortcut:** Press `Escape` to dismiss all visible toasts.

In [ ]:
# Alert demonstration callbacks - these show different alert types
def make_alert_toolbar():
    """Create a toolbar with buttons for each alert type."""
    return {
        "position": "top",
        "items": [
            {"type": "button", "label": "ℹ️ Info", "event": "alert:info"},
            {"type": "button", "label": "✅ Success", "event": "alert:success"},
            {"type": "button", "label": "⚠️ Warning", "event": "alert:warning"},
            {"type": "button", "label": "⛔ Error", "event": "alert:error"},
            {"type": "button", "label": "❓ Confirm", "event": "alert:confirm"},
        ],
    }


# Generic alert handlers that work with any widget
def create_alert_handlers(widget_name: str):
    """Create a set of alert handlers for a widget."""

    def show_info(data, event_type, label):
        widget = globals()[widget_name]
        widget.alert("Data refreshed successfully", alert_type="info")

    def show_success(data, event_type, label):
        widget = globals()[widget_name]
        widget.alert("Export complete!", alert_type="success", title="Done")

    def show_warning(data, event_type, label):
        widget = globals()[widget_name]
        widget.alert(
            "No items selected. Please select at least one row.",
            alert_type="warning",
            title="Selection Required",
        )

    def show_error(data, event_type, label):
        widget = globals()[widget_name]
        widget.alert(
            "Failed to connect to server. Please check your network.",
            alert_type="error",
            title="Connection Error",
        )

    def show_confirm(data, event_type, label):
        widget = globals()[widget_name]
        widget.alert(
            "Are you sure you want to delete these items?",
            alert_type="confirm",
            title="Confirm Delete",
            callback_event="alert:confirm-response",
        )

    def handle_confirm_response(data, event_type, label):
        widget = globals()[widget_name]
        if data.get("confirmed"):
            widget.alert("Items deleted successfully", alert_type="success")
        else:
            widget.alert("Deletion cancelled", alert_type="info")

    return {
        "alert:info": show_info,
        "alert:success": show_success,
        "alert:warning": show_warning,
        "alert:error": show_error,
        "alert:confirm": show_confirm,
        "alert:confirm-response": handle_confirm_response,
    }


print("Alert handlers defined for all rendering paths")

### 6.1 Alerts via AnyWidget

Click the toolbar buttons to see toast notifications. The `widget.alert()` method provides a convenient API.

In [ ]:
print("--- Alert Demo via AnyWidget ---")

alert_aw = app.show(
    "Hello from AnyWidget Alert Demo!",
    toolbars=[make_alert_toolbar()],
    callbacks=create_alert_handlers("alert_aw"),
    height=200,
)

In [ ]:
# Trigger an alert after the widget exists

alert_aw.alert("Hello from AnyWidget Alert Demo!", alert_type="confirm")

### 6.2 Alerts via IFrame

Same alert behavior works identically in the IFrame rendering path.

In [ ]:
print("--- Alert Demo via IFrame ---")

with force_iframe_mode():
    alert_iframe = app.show(
        "Hello from iFrame Alert Demo!",
        toolbars=[make_alert_toolbar()],
        callbacks=create_alert_handlers("alert_iframe"),
        height=200,
    )

### 6.3 Alerts via Native Window

Native windows also support the same alert API via `app.alert()`. The alerts appear as toast notifications inside the native window.

In [ ]:
print("--- Alert Demo via Native Window ---")


# Alert handlers for native window use app.alert() instead of widget.alert()
def show_info_native(data, event_type, label):
    app.alert("Data refreshed successfully", alert_type="info", label=label)


def show_success_native(data, event_type, label):
    app.alert("Export complete!", alert_type="success", title="Done", label=label)


def show_warning_native(data, event_type, label):
    app.alert(
        "No items selected. Please select at least one row.",
        alert_type="warning",
        title="Selection Required",
        label=label,
    )


def show_error_native(data, event_type, label):
    app.alert(
        "Failed to connect to server. Please check your network.",
        alert_type="error",
        title="Connection Error",
        label=label,
    )


def show_confirm_native(data, event_type, label):
    app.alert(
        "Are you sure you want to delete these items?",
        alert_type="confirm",
        title="Confirm Delete",
        callback_event="alert:confirm-response",
        label=label,
    )


def handle_confirm_response_native(data, event_type, label):
    if data.get("confirmed"):
        app.alert("Items deleted successfully", alert_type="success", label=label)
    else:
        app.alert("Deletion cancelled", alert_type="info", label=label)


native_alert_callbacks = {
    "alert:info": show_info_native,
    "alert:success": show_success_native,
    "alert:warning": show_warning_native,
    "alert:error": show_error_native,
    "alert:confirm": show_confirm_native,
    "alert:confirm-response": handle_confirm_response_native,
}

with force_native_window_mode():
    alert_native_label = app.show_plotly(
        figure=get_figure("plotly_dark"),
        title="Alert Demo (Native)",
        toolbars=[make_alert_toolbar()],
        callbacks=native_alert_callbacks,
    )

print(f"Native Alert demo window created: {alert_native_label}")